In [41]:
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from keras import models, Input
from keras import optimizers as opt
from keras import backend as K
from keras.layers import Dense
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import to_categorical

In [42]:
from dataset import load_dataset, load_labels, convert_to_epochs, format_labels
from features import time_series_features, fractal_features, entropy_features, hjorth_features, freq_band_features

# Variables

In [43]:
n_classes = 2
n_channels = 32
sfreq = 128

In [44]:
data_type = "ica_filtered"
test_type = "Arithmetic"

# Load Dataset

In [45]:
dataset_ = load_dataset(data_type=data_type, test_type=test_type)
dataset = convert_to_epochs(dataset_, n_channels, sfreq)

In [46]:
label_ = load_labels()
label = format_labels(label_, test_type=test_type, epochs=dataset.shape[1])

# Compute Features

In [47]:
features = time_series_features(dataset, n_channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = fractal_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features

# k-NN Classifier

In [48]:
x, x_test, y, y_test = train_test_split(
    data, label, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.25, random_state=1)
scaler = MinMaxScaler()
scaler.fit(x_train)
x = scaler.transform(x)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

param_grid = {
    'leaf_size': range(50),
    'n_neighbors': range(1, 10),
    'p': [1, 2]
}
split_index = [-1 if x in range(len(x_train)) else 0 for x in range(len(x))]
ps = PredefinedSplit(test_fold=split_index)
knn_clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=ps, refit=True)
knn_clf.fit(x, y)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': range(0, 50), 'n_neighbors': range(1, 10),
                         'p': [1, 2]})

In [49]:
y_pred = knn_clf.predict(x_test)
y_true = y_test

In [50]:
print(metrics.classification_report(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       False       0.66      0.60      0.63       311
        True       0.61      0.66      0.63       289

    accuracy                           0.63       600
   macro avg       0.63      0.63      0.63       600
weighted avg       0.63      0.63      0.63       600

[[188 123]
 [ 98 191]]


# SVM Classifier

In [51]:
x, x_test, y, y_test = train_test_split(
    data, label, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.25, random_state=1)

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
}
split_index = [-1 if x in range(len(x_train)) else 0 for x in range(len(x))]
ps = PredefinedSplit(test_fold=split_index)
svm_clf = GridSearchCV(SVC(), param_grid, cv=ps, refit=True)
svm_clf.fit(x, y)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']})

In [52]:
y_pred = svm_clf.predict(x_test)
y_true = y_test

In [53]:
print(metrics.classification_report(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       False       0.90      0.89      0.90       311
        True       0.88      0.90      0.89       289

    accuracy                           0.89       600
   macro avg       0.89      0.89      0.89       600
weighted avg       0.89      0.89      0.89       600

[[277  34]
 [ 30 259]]


# Multilayer Perceptron

In [54]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v)
x_train, x_test, y_train, y_test = train_test_split(
    data, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.25, random_state=1)

In [55]:
def model_builder(hp):
    model = models.Sequential()
    model.add(Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
        model.add(Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                        activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(Dense(n_classes, activation='softmax', name='out'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=opt.adam_v2.Adam(learning_rate=hp_learning_rate),
                  loss="binary_crossentropy",
                  metrics=['accuracy'])
    return model

In [56]:
tuner = RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    overwrite=True
)

In [57]:
tuner.search(x_train, y_train, epochs=50, validation_data=[x_val, y_val])

Trial 5 Complete [00h 00m 57s]
val_accuracy: 0.8333333432674408

Best val_accuracy So Far: 0.8658333420753479
Total elapsed time: 00h 04m 43s
INFO:tensorflow:Oracle triggered exit


In [58]:
model = tuner.get_best_models(num_models=1)[0]

In [59]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

2022-11-11 10:27:25.705634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [60]:
print(metrics.classification_report(y_true, y_pred))
print(metrics.confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       311
           1       0.87      0.82      0.85       289

    accuracy                           0.85       600
   macro avg       0.86      0.85      0.85       600
weighted avg       0.86      0.85      0.85       600

[[275  36]
 [ 51 238]]
